In [7]:
from langchain_groq import ChatGroq

In [ ]:
llm=ChatGroq(
    temperature=0,
    groq_api_key="YOUR API KEY PLEASE......",
    model_name="llama-3.1-70b-versatile"
)
response=llm.invoke("The First Person to land on Moon was . . .")
print(response.content)

The first person to land on the Moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [10]:
from langchain_community.document_loaders import WebBaseLoader

loader=WebBaseLoader("https://jobs.nike.com/job/R-45253")
page_data=loader.load().pop().page_content
print(page_data)

Apply for Data Engineer, ITC

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsData & AnalyticsData Engineer, ITCKarnataka, IndiaBecome a Part of the NIKE, Inc. Team
NIKE, Inc. does more 

In [11]:
pip install langchain_community

Note: you may need to restart the kernel to use updated packages.


In [23]:
from langchain_core.prompts import PromptTemplate

# Define the template
prompt_extract =PromptTemplate.from_template(
    """
Given the following job posting, extract the role, description, and skills in JSON format.

Job Posting:
{page_data}

Extracted Information:
"""
)

# Chain the prompt template with the language model (llm)
chain_extract = prompt_extract | llm

# Run the chain with the provided input
res = chain_extract.invoke(input={'page_data': page_data})
type(res.content)


str

In [27]:
from langchain_core.output_parsers import JsonOutputParser

json_parser=JsonOutputParser()
json_res=json_parser.parse(res.content)
json_res

{'role': 'Data Engineer, ITC',
 'description': "A data engineer will work in the Data and Artificial Intelligence organisation of Nike and will focus on building highly complex and performant data pipelines that'll drive Nike's data driven strategies for the future of sports.",
 'skills': ['Bachelor’s degree in computer science engineering',
  '2+ years of hands-on experience in data engineering field',
  'In depth big data tech stack knowledge',
  'Expertise in pyspark and SQL',
  'Expertise in databricks, snowflake, airflow',
  'Excellent written and verbal communication skills'],
 'responsibilities': ['Building, enhancing, and troubleshooting complex data pipelines',
  'Collaborating with product managers, engineers, analysts to build, enhance and troubleshoot data pipelines',
  'Collaborate with senior, lead and principal engineers to define and implement quality standards across data pipelines',
  'Contribute towards the design and architecture of data pipelines',
  'Implement dat

In [29]:
type(json_res)

dict

In [31]:
import pandas as pd

df=pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [35]:
import uuid
import chromadb

client=chromadb.PersistentClient("vectorstore")
collection=client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _,row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links":row["Links"]},
                       ids=[str(uuid.uuid4())])

In [46]:
# Query the collection with n_results set to 2
links = collection.query(query_texts=["Experience in Python", "Expertise in React"], n_results=2)

# Extract the 'metadatas' from the results
metadatas = links.get('metadatas')
metadatas


[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [48]:
job=json_res
job['skills']

['Bachelor’s degree in computer science engineering',
 '2+ years of hands-on experience in data engineering field',
 'In depth big data tech stack knowledge',
 'Expertise in pyspark and SQL',
 'Expertise in databricks, snowflake, airflow',
 'Excellent written and verbal communication skills']

In [50]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Data Engineering Solutions for Nike's Data-Driven Strategies

Dear Hiring Manager,

I came across the Data Engineer, ITC role at Nike and was impressed by the company's commitment to leveraging data-driven strategies for the future of sports. As a Business Development Executive at AtliQ, I'd like to introduce our team's expertise in delivering high-quality data engineering solutions that can support Nike's ambitious goals.

At AtliQ, we have a proven track record of designing and implementing complex data pipelines, leveraging cutting-edge technologies such as PySpark, SQL, Databricks, Snowflake, and Airflow. Our team of experienced data engineers has a deep understanding of big data tech stacks and is well-equipped to handle the challenges of building highly performant data pipelines.

We'd like to highlight a few examples of our previous work that demonstrate our capabilities:

* Machine Learning and Python-based solutions: https://example.com/ml-python-portfolio
* Py